#Load data

In [1]:
from google.colab import files
uploaded = files.upload()

Saving dialogues_018.json to dialogues_018.json


In [2]:
# intents
!mkdir -p dialogue_018/NONE
!mkdir -p dialogue_018/SearchOnewayFlight
!mkdir -p dialogue_018/ReserveOnewayFlight
!mkdir -p dialogue_018/SearchRoundtripFlights
!mkdir -p dialogue_018/ReserveRoundtripFlights

Script para coletar apenas as utterances (frases) do usuário e armazená-las num diretório que representa a intenção(que será nossa classe para a tarefa de classificação de intenção).</br>
<b>atenção</b>: O sistema foi excluído da coleta de utterances porque a resposta do sistema não possui intenção

In [3]:
import json

if __name__ == '__main__':
    with open("dialogues_018.json", 'r') as file:
        data_018 = json.load(file)

    for i in range(len(data_018)):
        frames_amount = len(data_018[i]['turns'])
        for j in range(frames_amount):
            current_frame = data_018[i]['turns'][j]
            speaker = current_frame['speaker']
            if speaker == 'USER':
                intent = current_frame['frames'][0]['state']['active_intent']
                utterance = current_frame['utterance']

                file_path = "dialogue_018/" + intent + "/" + str(i) + "_" + str(j) + ".txt"

                f = open(file_path, "w+")
                f.write(utterance)
                f.close()


Armazenar numa estrutura para treinamento no Keras

In [4]:
import tensorflow as tf

seed = 42

raw_train_ds= tf.keras.utils.text_dataset_from_directory(
    '/content/dialogue_018',
    labels='inferred',
    validation_split=0.2,
    subset='training',
    seed=seed,
    shuffle=False)

raw_validation_ds = tf.keras.utils.text_dataset_from_directory(
    '/content/dialogue_018',
    labels='inferred',
    validation_split=0.2,
    subset='validation',
    seed=seed,
    shuffle=False)

Found 1052 files belonging to 5 classes.
Using 842 files for training.
Found 1052 files belonging to 5 classes.
Using 210 files for validation.


In [5]:
raw_train_ds.class_names

['NONE',
 'ReserveOnewayFlight',
 'ReserveRoundtripFlights',
 'SearchOnewayFlight',
 'SearchRoundtripFlights']

In [ ]:
for text_batch, label_batch in raw_train_ds:
  print("Utterance: ", text_batch)
  print("Label: ", label_batch)

In [7]:
print(raw_train_ds)

<_BatchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


Acho que o Tensor acima é dividido em batches e acredito que isso adiciona uma complexidade na hora de tokenizar, pois teríamos que fazer para cada um dos tensores. Testei resolver fazer uma concatenação de todos os tensores abaixo:

In [8]:
all_text_tensor = tf.concat([text_batch for text_batch, label_batch in raw_train_ds], axis=0)
#all_text_tensor

In [9]:
all_labels = tf.concat([label_batch for text_batch, label_batch in raw_train_ds], axis=0)
#all_labels

In [10]:
utterances_validation_tensor = tf.concat([text_batch for text_batch, label_batch in raw_validation_ds], axis=0)

In [11]:
validation_labels = tf.concat([label_batch for text_batch, label_batch in raw_validation_ds], axis=0)

# Pre processamento de dados
Fazer uma bag-of-words

In [12]:
from keras.preprocessing.text import Tokenizer
import json
import numpy
from keras.preprocessing import text, sequence

### Train subset

In [13]:
t = Tokenizer()
texts = all_text_tensor.numpy().astype(str)
t.fit_on_texts(texts)

# summarize what was learned
print(t.word_counts)
print(t.document_count)
print(t.word_index)
print(t.word_docs)
# integer encode documents
encoded_docs = t.texts_to_matrix(texts, mode='count')
print(encoded_docs)

OrderedDict([('no', 73), ('that', 175), ('will', 62), ('be', 70), ('all', 75), ('thank', 75), ('you', 191), ("that's", 68), ('thanks', 92), ('for', 217), ('your', 54), ('help', 101), ('nope', 8), ("we're", 1), ('good', 101), ('a', 93), ('lot', 5), ('the', 280), ('is', 114), ("i'm", 46), ('right', 17), ('though', 1), ('thats', 8), ('it', 54), ('i', 282), ('just', 8), ('needed', 8), ('information', 2), ('now', 11), ('again', 4), ('everything', 13), ('would', 50), ('very', 10), ('much', 16), ('do', 17), ('not', 5), ('want', 54), ('yet', 1), ('was', 1), ('okay', 11), ('this', 63), ('should', 1), ('enough', 1), ('to', 225), ('know', 4), ('maybe', 1), ('another', 6), ('time', 24), ("that'll", 1), ('wanted', 2), ('but', 6), ('am', 62), ('nothing', 1), ('else', 7), ('set', 1), ('yes', 122), ('can', 93), ('assist', 3), ('me', 145), ('with', 14), ('buying', 3), ('ticket', 67), ('correct', 20), ('yeah', 3), ('perfect', 24), ('great', 34), ('sure', 2), ('go', 16), ('ahead', 4), ('and', 125), ('buy

In [14]:
seq_token = t.texts_to_sequences(texts)
padded_sequences = sequence.pad_sequences(seq_token)

In [15]:
padded_sequences.shape

(842, 30)

In [16]:
all_labels.shape

TensorShape([842])

### Validation subset

In [17]:
t2 = Tokenizer()
validation_texts = utterances_validation_tensor.numpy().astype(str)
t2.fit_on_texts(validation_texts)
validation_seq_token = t2.texts_to_sequences(validation_texts)
padded_validation = sequence.pad_sequences(validation_seq_token)
#padded_validation = sequence.pad_sequences(validation_seq_token, maxlen=30)

In [18]:
padded_validation.shape

(210, 28)

In [19]:
validation_labels.shape

TensorShape([210])

# Classificação de intenções (Parte 1)
https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM </br>
https://keras.io/examples/nlp/text_classification_with_transformer/ </br>

## LSTM

In [ ]:
from keras.models import Sequential
from keras.layers import Input, Dense, LSTM
from keras.layers import Embedding
import keras.backend as K

model_lstm = Sequential()
model_lstm.add(Embedding(input_dim=842, output_dim=100))
model_lstm.add(LSTM(units=64))
model_lstm.add(Dense(5,activation='softmax'))
model_lstm.compile(loss=['sparse_categorical_crossentropy'] , optimizer='adam', metrics=['accuracy'])
model_lstm.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, None, 100)         84200     
                                                                 
 lstm_1 (LSTM)               (None, 64)                42240     
                                                                 
 dense_25 (Dense)            (None, 5)                 325       
                                                                 
Total params: 126765 (495.18 KB)
Trainable params: 126765 (495.18 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history_lstm = model_lstm.fit(padded_sequences, all_labels, epochs=10)

Epoch 1/10
27/27 [==============================] - 3s 22ms/step - loss: 1.3791 - accuracy: 0.4513
Epoch 2/10
27/27 [==============================] - 1s 23ms/step - loss: 1.1015 - accuracy: 0.6306
Epoch 3/10
27/27 [==============================] - 1s 23ms/step - loss: 0.8571 - accuracy: 0.7067
Epoch 4/10
27/27 [==============================] - 1s 24ms/step - loss: 0.6919 - accuracy: 0.7494
Epoch 5/10
27/27 [==============================] - 1s 23ms/step - loss: 0.5871 - accuracy: 0.7933
Epoch 6/10
27/27 [==============================] - 1s 23ms/step - loss: 0.5115 - accuracy: 0.8349
Epoch 7/10
27/27 [==============================] - 1s 22ms/step - loss: 0.4486 - accuracy: 0.8551
Epoch 8/10
27/27 [==============================] - 1s 23ms/step - loss: 0.4092 - accuracy: 0.8504
Epoch 9/10
27/27 [==============================] - 1s 26ms/step - loss: 0.3609 - accuracy: 0.8741
Epoch 10/10
27/27 [==============================] - 1s 39ms/step - loss: 0.3231 - accuracy: 0.8931


In [ ]:
history_with_validation_lstm = model_lstm.fit(padded_sequences, all_labels, validation_data=(padded_validation, validation_labels), epochs=10)

Epoch 1/10
27/27 [==============================] - 1s 48ms/step - loss: 0.2933 - accuracy: 0.9050 - val_loss: 2.8053 - val_accuracy: 0.2952
Epoch 2/10
27/27 [==============================] - 1s 25ms/step - loss: 0.2887 - accuracy: 0.9026 - val_loss: 2.6944 - val_accuracy: 0.3143
Epoch 3/10
27/27 [==============================] - 1s 26ms/step - loss: 0.2601 - accuracy: 0.9169 - val_loss: 2.4873 - val_accuracy: 0.3381
Epoch 4/10
27/27 [==============================] - 1s 26ms/step - loss: 0.2324 - accuracy: 0.9216 - val_loss: 2.0444 - val_accuracy: 0.4762
Epoch 5/10
27/27 [==============================] - 1s 27ms/step - loss: 0.2200 - accuracy: 0.9287 - val_loss: 2.4590 - val_accuracy: 0.3952
Epoch 6/10
27/27 [==============================] - 1s 25ms/step - loss: 0.2256 - accuracy: 0.9169 - val_loss: 2.4315 - val_accuracy: 0.3714
Epoch 7/10
27/27 [==============================] - 1s 26ms/step - loss: 0.2115 - accuracy: 0.9264 - val_loss: 2.9085 - val_accuracy: 0.3190
Epoch 8/10
27

## SVM

In [ ]:
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

with open("dialogues_018.json", 'r') as file:
    data_018 = json.load(file)

texts = []
labels = []

for i in range(len(data_018)):
    frames_amount = len(data_018[i]['turns'])
    for j in range(frames_amount):
        current_frame = data_018[i]['turns'][j]
        speaker = current_frame['speaker']
        if speaker == 'USER':
            intent = current_frame['frames'][0]['state']['active_intent']
            utterance = current_frame['utterance']
            texts.append(utterance)
            labels.append(intent)

texts_train, texts_validation, labels_train, labels_validation = train_test_split(texts, labels, test_size=0.2, random_state=42)

vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(texts_train)
X_validation = vectorizer.transform(texts_validation)

svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, labels_train)

predictions = svm_model.predict(X_validation)

accuracy = accuracy_score(labels_validation, predictions)
print(f'Acurácia do modelo SVM: {accuracy}')


Acurácia do modelo SVM: 0.8104265402843602


In [ ]:
# Exemplo de diálogo
example_dialogue = [
    {"speaker": "USER", "utterance": "I would like a short trip to New York"}
    # Adicione mais turnos do diálogo conforme necessário
]

# Pré-processamento do exemplo de diálogo
example_text = example_dialogue[0]['utterance']
example_vector = vectorizer.transform([example_text])

# Previsão do modelo SVM
predicted_intent = svm_model.predict(example_vector)

# Resultados
print(f"Texto do Usuário: {example_text}")
print(f"Intenção Prevista: {predicted_intent[0]}")


Texto do Usuário: I would like a short trip to New York
Intenção Prevista: SearchRoundtripFlights


## CNN

In [ ]:
import pandas as pd
import numpy as np
import time

from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.preprocessing.text import Tokenizer, one_hot
from keras.preprocessing.sequence import pad_sequences

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

model = Sequential()
model.add(layers.Embedding(input_dim=842, output_dim=100))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Alterado para categorical_crossentropy
              metrics=['accuracy'])

print(model.summary())

# Convertendo os rótulos para codificação one-hot
from tensorflow.keras.utils import to_categorical
labels_one_hot = to_categorical(all_labels, num_classes=5)

# Fit do modelo
history = model.fit(padded_sequences, labels_one_hot, epochs=10, validation_data=(padded_validation, to_categorical(validation_labels, num_classes=5)), batch_size=5)


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, None, 100)         84200     
                                                                 
 conv1d_8 (Conv1D)           (None, None, 128)         64128     
                                                                 
 global_max_pooling1d_8 (Gl  (None, 128)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_16 (Dense)            (None, 64)                8256      
                                                                 
 dense_17 (Dense)            (None, 5)                 325       
                                                                 
Total params: 156909 (612.93 KB)
Trainable params: 156909 (612.93 KB)
Non-trainable params: 0 (0.00 Byte)
______________

In [ ]:
# Supondo que você tenha um novo texto a ser classificado
new_text = "i need to buy a one way flight"

new_sequence = t.texts_to_sequences([new_text])
padded_new_sequence = pad_sequences(new_sequence, maxlen=10, padding='post')

predictions = model.predict(padded_new_sequence)

predicted_class = predictions.argmax(axis=-1)

print(f'O texto foi classificado como classe: {predicted_class[0]}')


1/1 [==============================] - 0s 115ms/step
O texto foi classificado como classe: 1


## Transformer
implementar incialmente exatamente como está na documentação e depois testar mudar os parâmetros

### Teste de transformer com BERT

In [20]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.layers import Dense

In [22]:
# Load BERT tokenizer and model
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
# Modify the model's classifier layer
classifier_layer = Dense(5, activation='softmax',  name='classifier')
bert_model.layers[-1]= classifier_layer

In [32]:
import keras
opt = keras.optimizers.Adam(learning_rate=2e-5)
bert_model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [29]:
bert_history = bert_model.fit(
    padded_sequences,
    all_labels,
    epochs=3
)

Epoch 1/3
27/27 [==============================] - 328s 11s/step - loss: 3.1591 - accuracy: 0.3753
Epoch 2/3
27/27 [==============================] - 279s 10s/step - loss: 1.9506 - accuracy: 0.3717
Epoch 3/3
27/27 [==============================] - 283s 10s/step - loss: 1.4578 - accuracy: 0.4204
